In [64]:
from bs4 import BeautifulSoup 
from selenium import webdriver
# The following packages will also be used in this tutorial
import pandas as pd
import numpy as np
import time
import requests
import re
from sklearn.feature_extraction.text import CountVectorizer
from joblib import Parallel, delayed


"""
    Getting started
"""
def getPage(url):
    ''' returns a soup object that contains all the information 
    of a certain webpage'''
    result = requests.get(url)
    content = result.content
    return BeautifulSoup(content, features = "lxml")

    
def getRoomClasses(soupPage):
    ''' This function returns all the listings that can 
    be found on the page in a list.'''
    rooms = soupPage.findAll("div", {"class": "_8ssblpx"})
    result = []
    for room in rooms:
        result.append(room)
    return result
La0 = "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c"
La1 = "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c&items_offset=20&section_offset=3"
La2 =  "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c&items_offset=40&section_offset=3"
La3 = "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c&items_offset=60&section_offset=3"
La4 =  "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c&items_offset=80&section_offset=3"
La5 = "https://www.airbnb.com/s/Los-Angeles--CA--United-States/homes?adults=2&source=structured_search_input_header&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJE9on3F3HwoAR9AhGJW_fL-I&federated_search_session_id=bb424b64-5dd3-4c12-bf29-a3ce9f86456c&items_offset=100&section_offset=3"  
urls =[La0,La2,La3,La4,La5] 


In [5]:
def getTopRow(listing):
    ''' Returns the top row of listing information'''
    return listing.find("div", {"class": "_167qordg"}).text
def getRoomInfo(listing):
    ''' Returns the guest information'''
    return listing.find("div", {"class":"_kqh46o"}).text
def getBasicFacilities(listing):
    ''' Returns the basic facilities'''
    try:
        output = listing.findAll("div", {"class":"_kqh46o"})[1].text.replace(" ","") #Speeds up cleaning
    except:
        output = []
    return output
def getListingPrice(listing):
    ''' Returns the price'''
    return listing.find("div", {"class":"_1fwiw8gv"}).text

def getListingRating(listing):
    ''' Returns the rating '''
    return listing.find("span", {"class":"_krjbj"}).text

def getListingReviewNumber(listing):
    ''' Returns the number of reviews '''
    try: # Not all listings have reviews // extraction failed
        output = listing.findAll("span", {"class":"_krjbj"})[1].text
    except:
        output = -1   # Indicate that the extraction failed -> can indicate no reviews or a mistake in scraping
    return output

In [6]:
def extractInformation(soupPage):
    ''' Takes all the information of a single page (thus multiple listings) and
    summarizes it in a dataframe'''
    listings = getRoomClasses(soupPage)
    toprows, roominfos, basicfacilitiess, prices, ratings, reviews = [], [], [], [], [], []
    for listing in listings:
        toprows.append(getTopRow(listing))
        roominfos.append(getRoomInfo(listing))
        basicfacilitiess.append(getBasicFacilities(listing))
        prices.append(getListingPrice(listing))
        ratings.append(getListingRating(listing))
        reviews.append(getListingReviewNumber(listing))
    dictionary = {"RoomType": toprows, "roominfo": roominfos, "facilities" : basicfacilitiess, "price": prices, "rating": ratings, "reviewnumber": reviews}
    return pd.DataFrame(dictionary)

In [44]:
def getPages(urls):
    ''' This function returns all the links to the pages containing 
    listings for one particular city '''
    result = []
    for url in urls:
        page = getPage(url)
        result = result + [page]
    return result


In [60]:
def extractPages(url):
    ''' This function outputs a dataframe that contains all information of a particular
    city. It thus contains information of multiple listings coming from multiple pages.'''
    pages = getPages(urls)
    # Do for the first element to initialize the dataframe
    df = extractInformation(pages[0])
    # Loop over all other elements of the dataframe
    for pagenumber in range(1, len(pages)):
        df = df.append(extractInformation(pages[pagenumber]))
    return df

In [62]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [316]:
airbnb_dub =extractPages(dubUrls)


# Data Cleaning

In [317]:
print(airbnb_dub.shape)
airbnb_dub.columns

(100, 6)


Index(['RoomType', 'roominfo', 'facilities', 'price', 'rating',
       'reviewnumber'],
      dtype='object')

In [319]:
airbnb_dub.loc[:, "guests"] = airbnb_dub.loc[:, "roominfo"].str.split(" · ", n = 0, expand = True)[0].str.replace(" guests", "")
airbnb_dub.loc[:, "bedrooms"] = airbnb_dub.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[1]
airbnb_dub.loc[:, "beds"] = airbnb_dub.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[2].str.replace(" bed", "").str.replace("s", "")
airbnb_dub.loc[:, "bathrooms"] = airbnb_dub.loc[:, "roominfo"].str.split(" . ", n = 0, expand = True)[3]


In [320]:
airbnb_dub.loc[:, "guests"] = pd.to_numeric(airbnb_dub.guests, errors = 'coerce')
airbnb_dub.loc[:, "beds"] = pd.to_numeric(airbnb_dub.beds, errors = 'coerce')
airbnb_dub.loc[:, "bedrooms"] = pd.to_numeric(airbnb_dub.bedrooms.str.split(" ", n = 0, expand = True)[0], errors = "ignore")
airbnb_dub.loc[:, "bathrooms"] = pd.to_numeric(airbnb_dub.bathrooms.str.split(" ", n = 0, expand = True)[0], errors = "ignore")



In [321]:
airbnb_dub['bathrooms']=airbnb_dub['bathrooms'].replace(np.nan, 1)
airbnb_dub['bedrooms']=airbnb_dub['bedrooms'].replace('Studio', 1)

airbnb_dub['bedrooms']=airbnb_dub['bedrooms'].replace(np.nan, 1)
airbnb_dub['beds'] =airbnb_dub['beds'].replace(np.nan,1)
airbnb_dub['guests'] =airbnb_dub['guests'].replace(np.nan,2)

## price

In [322]:
Price=[]
for x in airbnb_dub["price"]:
    y =re.search(r'\d+', x).group(0)
    Price.append(int(y))

In [323]:
airbnb_dub['price']=Price

## rating

In [324]:
airbnb_dub.loc[:, "score"] = airbnb_dub.loc[:, 'rating'].str.split(" ", n = 0, expand = True)[1]
airbnb_dub['score']=airbnb_dub['score'].replace(np.nan, 0)
airbnb_dub.loc[:, "score"] = pd.to_numeric(airbnb_tok.score, errors = "coerce")


## review

In [325]:
airbnb_dub.loc[:, "reviews"] = airbnb_dub.loc[:, 'reviewnumber'].str.split(" ", n = 0, expand = True)[0]
airbnb_dub['reviews']=airbnb_dub['reviews'].replace(np.nan, 0)  


## facilities

In [327]:
airbnb_dub.loc[:, "facilities"] = airbnb_dub["facilities"].astype(str).str.replace("[","").str.replace("]","")
vectorizer = CountVectorizer(decode_error = "ignore") 
X = vectorizer.fit_transform(airbnb_dub.facilities)
bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())


In [328]:
airbnb_dub=pd.concat([airbnb_dub.reset_index(drop=True).drop("facilities", axis = 1), bag_of_words], axis=1)


In [329]:
airbnb_dub.columns


Index(['RoomType', 'roominfo', 'price', 'rating', 'reviewnumber', 'guests',
       'bedrooms', 'beds', 'bathrooms', 'score', 'reviews', 'airconditioning',
       'beachfront', 'breakfast', 'dishwasher', 'dryer', 'elevator',
       'freeparking', 'gym', 'hottub', 'in', 'indoorfireplace', 'kitchen',
       'petsallowed', 'pool', 'selfcheck', 'washer', 'wifi'],
      dtype='object')

In [330]:
airbnb_dub=airbnb_dub.drop(['RoomType','roominfo', 'gym','rating','hottub','breakfast','dishwasher', 'reviewnumber','selfcheck','dryer','pool', 'washer', 'in','indoorfireplace'], axis =1)


In [332]:
airbnb_dub=airbnb_dub.drop(['beachfront'],axis =1)

In [333]:
airbnb_dub['city'] ='dubai'
airbnb_dub['country'] = 'UnitedArabEmirates'
airbnb_dub.columns
print(airbnb_dub.shape)

(100, 15)


In [194]:
#airbnb_lond=airbnb_lond.drop(['beachfront'],axis=1)

In [334]:
airbnb_dub.reset_index(drop=True)

,price,guests,bedrooms,beds,bathrooms,score,reviews,airconditioning,elevator,freeparking,kitchen,petsallowed,wifi,city,country
0,44,2,1,1.0,1.0,4.81,180,0,0,0,1,0,1,dubai,UnitedArabEmirates
1,56,4,1,2.0,2.5,0.00,0,0,0,0,0,0,1,dubai,UnitedArabEmirates
2,94,6,1,3.0,1.0,4.95,0,0,0,0,1,0,1,dubai,UnitedArabEmirates
3,56,2,1,1.0,1.0,0.00,0,0,0,1,1,0,1,dubai,UnitedArabEmirates
4,60,4,1,1.0,1.0,0.00,0,0,0,1,1,0,1,dubai,UnitedArabEmirates
5,58,3,1,1.0,1.0,0.00,0,0,0,0,1,0,1,dubai,UnitedArabEmirates
6,48,2,1,1.0,1.0,0.00,0,0,0,1,1,0,1,dubai,UnitedArabEmirates
7,52,4,1,2.0,1.0,0.00,0,0,0,0,1,0,1,dubai,UnitedArabEmirates
8,71,3,1,2.0,2.5,0.00,0,0,0,1,1,0,1,dubai,UnitedArabEmirates
9,62,2,1,1.0,1.0,0.00,0,0,0,0,1,0,1,dubai,UnitedArabEmirates


In [335]:
#airbnb.reset_index(drop=True)
airbnb_dub.to_csv('dubai.csv')

In [137]:
ny0="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e"
ny1="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e&items_offset=20&section_offset=3"
ny2="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e&items_offset=40&section_offset=3"
ny3="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e&items_offset=60&section_offset=3"
ny4="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e&items_offset=80&section_offset=3"
ny5="https://www.airbnb.com/s/New-York--NY--United-States/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJOwg_06VPwokRYv534QaPC8g&federated_search_session_id=d6ccbc38-1a18-4a15-86f0-757bf22d1a4e&items_offset=100&section_offset=3"
nyUrls=[ny0,ny1,ny2,ny3,ny4,ny5]


In [169]:
ldon0="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b"
ldon1="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b&items_offset=20&section_offset=3"
ldon2="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b&items_offset=40&section_offset=3"
ldon3="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b&items_offset=60&section_offset=3"
ldon4="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b&items_offset=80&section_offset=3"
ldon5="https://www.airbnb.com/s/London--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJdd4hrwug2EcRmSrV3Vo6llI&federated_search_session_id=ad46b4ca-ab0f-4843-9410-a04e050da79b&items_offset=100&section_offset=3"

ldonUrls=[ldon0,ldon1,ldon2,ldon3,ldon4,ldon5]


In [197]:
birm0="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7"
birm1="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7&items_offset=20&section_offset=3"
birm2="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7&items_offset=40&section_offset=3"
birm3="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7&items_offset=60&section_offset=3"
birm4="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7&items_offset=80&section_offset=3"
birm5="https://www.airbnb.com/s/Birmingham--United-Kingdom/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJc3FBGy2UcEgRmHnurvD-gco&federated_search_session_id=22f4c9ac-4b68-405d-be73-4e67eeedb5b7&items_offset=100&section_offset=3"
birmUrls=[birm0,birm1,birm2,birm3,birm4,birm5]


In [225]:
par0="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198"
par1="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198&items_offset=20&section_offset=3"
par2="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198&items_offset=40&section_offset=3"
par3="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198&items_offset=60&section_offset=3"
par4 ="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198&items_offset=80&section_offset=3"
par5="https://www.airbnb.com/s/Paris--France/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJD7fiBh9u5kcRYJSMaMOCCwQ&federated_search_session_id=1391d093-acb5-4293-88c1-776cd6163198&items_offset=100&section_offset=3"
parUrls=[par0,par1,par2,par3,par4,par5]


In [243]:
hk0 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8"
hk1 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8&items_offset=20&section_offset=3"
hk2 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8&items_offset=40&section_offset=3"
hk3 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8&items_offset=60&section_offset=3"
hk4 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8&items_offset=80&section_offset=3"
hk5 ="https://www.airbnb.com/s/Hongkong--Hong-Kong/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJByjqov3-AzQR2pT0dDW0bUg&federated_search_session_id=790bd406-9555-4896-96bc-fb1d204283a8&items_offset=100&section_offset=3"
hkUrls=[hk0,hk1,hk2,hk3,hk4,hk5]



In [261]:
bar0 ="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591"
bar1="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591&items_offset=20&section_offset=3"
bar2="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591&items_offset=40&section_offset=3"
bar3="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591&items_offset=60&section_offset=3"
bar4="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591&items_offset=80&section_offset=3"
bar5="https://www.airbnb.com/s/Barcelona--Spain/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ5TCOcRaYpBIRCmZHTz37sEQ&federated_search_session_id=fa676390-c04b-4424-bbf4-e4b3d8726591&items_offset=100&section_offset=3"
barUrls =[bar0,bar1,bar2,bar3, bar4, bar5]



In [279]:
bak0 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f"
bak1 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f&items_offset=20&section_offset=3"
bak2 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f&items_offset=40&section_offset=3"
bak3 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f&items_offset=60&section_offset=3"
bak4 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f&items_offset=80&section_offset=3"
bak5 ="https://www.airbnb.com/s/Bangkok-Thailand/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ82ENKDJgHTERIEjiXbIAAQE&federated_search_session_id=6a7ad9ec-0acf-4844-8b78-af3ff696737f&items_offset=100&section_offset=3"
bakUrls =[bak0,bak2,bak3,bak4,bak5]





In [296]:
tok0 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d"
tok1 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d&items_offset=20&section_offset=5"
tok2 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d&items_offset=40&section_offset=5"
tok3 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d&items_offset=60&section_offset=5"
tok4 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d&items_offset=80&section_offset=5"
tok5 ="https://www.airbnb.com/s/Tokyo--Japan/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJ51cu8IcbXWARiRtXIothAS4&federated_search_session_id=128fbc6d-7641-45f8-b257-a4c5b840e59d&items_offset=100&section_offset=5"
tokUrls =[tok0,tok1,tok2,tok3,tok4,tok5]





In [315]:
dua0 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67"
dua1 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67&items_offset=20&section_offset=3"
dua2 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67&items_offset=40&section_offset=3"
dua3 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67&items_offset=60&section_offset=3"
dua4 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67&items_offset=80&section_offset=3"
dua5 ="https://www.airbnb.com/s/Dubai-~-United-Arab-Emirates/homes?adults=2&source=search_blocks_selector_p1_flow&search_type=pagination&tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&place_id=ChIJRcbZaklDXz4RYlEphFBu5r0&federated_search_session_id=5782c8c2-18fd-4d26-9857-574180343b67&items_offset=100&section_offset=3"
dubUrls=[dua0,dua1,dua2,dua3,dua4,dua5]





In [400]:
london.to_csv('london.csv')  
Newyork.to_csv('ny.csv')  
Bangkok.to_csv('bangkok.csv') 
Birmingham.to_csv('brimingham.csv')
Barcelona.to_csv('barcelona.csv')
Losangles.to_csv('LA.csv')
Dubai.to_csv('dubai.csv')
Hongkong.to_csv('hongkong.csv')
Pais.to_csv('paris.csv')

In [401]:
print(london.shape)
print(Newyork.shape)
print(Bangkok.shape)
print(Birmingham.shape)
print(Barcelona.shape)
print(Losangles.shape)

Birmingham.columns

(100, 15)
(100, 15)
(100, 15)
(100, 15)
(100, 15)
(100, 15)


Index(['Unnamed: 0', 'price', 'guests', 'bedrooms', 'beds', 'bathrooms',
       'score', 'reviews', 'airconditioning', 'freeparking', 'kitchen',
       'petsallowed', 'wifi', 'city', 'country'],
      dtype='object')

(100, 15)


In [361]:
Losangles.columns

Index(['Unnamed: 0', 'price', 'guests', 'bedrooms', 'beds', 'bathrooms',
       'airconditioning', 'elevator', 'freeparking', 'indoorfireplace',
       'kitchen', 'petsallowed', 'wifi', 'score', 'city', 'country',
       'reviews'],
      dtype='object')

In [402]:
frames = [london, Newyork, Bangkok,Birmingham,Barcelona,Losangles]

In [403]:
result = pd.concat(frames)

In [405]:
result.to_csv('overall.csv')

In [407]:
df=result.groupby('city').agg(["mean"]).reset_index()


In [412]:
DF=result.groupby('country').agg(["mean"]).reset_index()


,country,Unnamed: 0,price,guests,bedrooms,beds,score,reviews,airconditioning,freeparking,kitchen,petsallowed,wifi
,,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
0,England,49.5,73.98,2.885,1.120,1.450,0.4905,71.405,0.65,0.35,0.74,0.225,0.99
1,Japan,49.5,67.07,2.910,1.110,1.380,2.6914,87.000,0.67,0.25,0.71,0.220,1.00
2,Spain,49.5,83.13,2.770,1.030,1.360,3.1474,99.250,0.48,0.25,0.63,0.090,1.00
3,US,49.5,75.02,2.945,1.135,1.475,1.3122,70.290,0.24,0.47,0.84,0.045,0.98
